<a href="https://colab.research.google.com/github/hasnil/bug-or-nug/blob/main/Colab_Notebook_BUG_or_NUG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 0. Importing Modules and Installations

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java

!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

update-alternatives: using /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java to provide /usr/bin/java (java) in manual mode


In [ ]:
%%shell 
pip install nltk
pip install --upgrade pip
python -m nltk.downloader all
pip install bs4
pip install mlflow
pip install contractions
pip install spark-stratifier
pip install pyspark==3.3.0

In [4]:
from nltk.stem.porter import *
from pyspark.sql.functions import col, lower, trim, regexp_replace, concat_ws, udf, lit, array, row_number
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, CountVectorizer
from bs4 import BeautifulSoup
import contractions
from pyspark.sql import Row
from collections import Counter
from pyspark.sql.types import StringType, ArrayType, FloatType, IntegerType, StructType, StructField
import pandas
import numpy
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.classification import LinearSVC
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import udf
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import IndexToString
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import Word2Vec
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import Normalizer
from spark_stratifier import StratifiedCrossValidator
from pyspark.ml.feature import HashingTF, IDF
from pyspark.sql import SparkSession
import findspark

In [5]:
findspark.init()

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

spark

# 1. Examining Data

## 1.1 Examining Original Dataset

In [6]:
file_location = 'https://raw.githubusercontent.com/hasnil/bug-or-nug/1b30a4efae301aa6c13eebbad339f14e11a1dc74/Data%20As%20CSVs/OriginalPaperDataTypeColumnFixed.csv'
df_csv = pandas.read_csv(file_location)
temp_file_location = '/content/drive/My Drive/Colab Notebooks/OriginalPaperDataTypeColumnFixed.csv'
df_csv.to_csv(temp_file_location)


df_issues_original = spark.read.option("header", True).csv(temp_file_location).cache()

#Trim CLASSIFIED column
df_issues_original = df_issues_original.withColumn('CLASSIFIED', trim(df_issues_original.CLASSIFIED))

In [7]:
num_bug = df_issues_original.filter(col('CLASSIFIED') == 'BUG').count()
num_nug = df_issues_original.filter(col('CLASSIFIED') == 'NUG').count()
 
print('Total number of records:', df_issues_original.count())
print('Total number of bugs:', num_bug)
print('Total number of nugs:', num_nug)

Total number of records: 5589
Total number of bugs: 1940
Total number of nugs: 3649


In [8]:
def print_project_data_info(df, id_string, project_name):
  df_specific_project = df.filter(col("ID").contains(id_string))
  num_bugs = df_specific_project.filter(col('CLASSIFIED') == 'BUG').count()
  num_nugs = df_specific_project.filter(col('CLASSIFIED') == 'NUG').count()
  num_missing_labels = df_specific_project.filter(col("Type").isNull()).count()
  num_misclassified_as_nug = df_specific_project.filter(col("Type")=="NUG").filter(col("CLASSIFIED")=="BUG").count()
  num_misclassified_as_bug = df_specific_project.filter(col("Type")=="BUG").filter(col("CLASSIFIED")=="NUG").count()
  total_error_percent = (num_missing_labels + num_misclassified_as_nug + num_misclassified_as_bug)/df_specific_project.count()
  
  print('Number of', project_name, 'reports:', df_specific_project.count())
  print(project_name, 'number of bugs:', num_bugs)
  print(project_name,'number of nugs:', num_nugs)
  print('Number of', project_name, 'missing labels:', num_missing_labels)
  print('Number of', project_name, 'misclassified as NUG', num_misclassified_as_nug)
  print('Number of', project_name, 'misclassified as BUG', num_misclassified_as_bug)
  print(project_name, "total error percent:", total_error_percent)
  print()
 
  
print_project_data_info(df_issues_original, "HTTPCLIENT", "HTTPCLIENT")
print_project_data_info(df_issues_original, "JCR", "JACKRABBIT")
print_project_data_info(df_issues_original, "LUCENE", "LUCENE")

Number of HTTPCLIENT reports: 745
HTTPCLIENT number of bugs: 305
HTTPCLIENT number of nugs: 440
Number of HTTPCLIENT missing labels: 14
Number of HTTPCLIENT misclassified as NUG 6
Number of HTTPCLIENT misclassified as BUG 171
HTTPCLIENT total error percent: 0.2563758389261745

Number of JACKRABBIT reports: 2402
JACKRABBIT number of bugs: 938
JACKRABBIT number of nugs: 1464
Number of JACKRABBIT missing labels: 74
Number of JACKRABBIT misclassified as NUG 26
Number of JACKRABBIT misclassified as BUG 302
JACKRABBIT total error percent: 0.16736053288925895

Number of LUCENE reports: 2442
LUCENE number of bugs: 697
LUCENE number of nugs: 1745
Number of LUCENE missing labels: 61
Number of LUCENE misclassified as NUG 19
Number of LUCENE misclassified as BUG 358
LUCENE total error percent: 0.17936117936117937



In [9]:
def get_top_10_words_preliminary(df, label):
  all_words_for_reports_string = df.filter(col('CLASSIFIED') == label).select('SUMMARY').rdd.flatMap(lambda x: x).collect()
  temp = dict(Counter(" ".join(all_words_for_reports_string).split()))
  word_count = sorted(temp, key = temp.get, reverse = True)
  return word_count[0:10]
  
print("Top 10 Terms for BUGs", get_top_10_words_preliminary(df_issues_original, "BUG"))
print("Top 10 Terms for NUGs", get_top_10_words_preliminary(df_issues_original, "NUG"))

Top 10 Terms for BUGs ['in', 'not', 'with', 'the', 'to', 'when', 'a', 'of', 'is', 'does']
Top 10 Terms for NUGs ['to', 'in', 'for', 'of', 'and', 'the', 'should', 'a', 'Add', 'not']


## 1.2 Examining Newly Added Data

In [10]:
file_location = 'https://raw.githubusercontent.com/hasnil/bug-or-nug/main/Data%20As%20CSVs/AllNewDataCollected.csv'
df_csv = pandas.read_csv(file_location)
temp_file_location = '/content/drive/My Drive/Colab Notebooks/AllNewDataCollected.csv'
df_csv.to_csv(temp_file_location)


df_new_issues = spark.read.option("header", True).csv(temp_file_location).cache()
 
#Trim CLASSIFIED column
df_new_issues = df_new_issues.withColumn('CLASSIFIED', trim(df_new_issues.CLASSIFIED))

In [11]:
# Check for any null summary
df_null = df_new_issues.filter(col("SUMMARY").isNull())
print(df_null.count())

0


In [12]:
num_bug = df_new_issues.filter(col('CLASSIFIED') == 'BUG').count()
num_nug = df_new_issues.filter(col('CLASSIFIED') == 'NUG').count()
 
print('Total number of new records:', df_new_issues.count())
print('Number of bugs in new data:', num_bug)
print('Number of nugs in new data:', num_nug)
print()
 
print("Top 10 Terms for BUGs", get_top_10_words_preliminary(df_new_issues, "BUG"))
print("Top 10 Terms for NUGs", get_top_10_words_preliminary(df_new_issues, "NUG"))

Total number of new records: 800
Number of bugs in new data: 404
Number of nugs in new data: 396

Top 10 Terms for BUGs ['in', 'to', 'with', 'on', 'the', 'when', 'not', 'is', 'for', 'a']
Top 10 Terms for NUGs ['for', 'to', 'in', 'and', 'the', 'of', 'Add', 'a', 'on', 'support']


In [13]:
df_mongodb = df_new_issues.filter(col('ID').contains('SERVER'))
df_spark = df_new_issues.filter(col('ID').contains('SPARK'))
df_wildfly = df_new_issues.filter(col('ID').contains('WFLY'))
 
print_project_data_info(df_new_issues, "SERVER", "MongoDB")
print_project_data_info(df_new_issues, "SPARK", "Apache Spark")
print_project_data_info(df_new_issues, "WFLY", "WildFly")

Number of MongoDB reports: 342
MongoDB number of bugs: 157
MongoDB number of nugs: 185
Number of MongoDB missing labels: 0
Number of MongoDB misclassified as NUG 6
Number of MongoDB misclassified as BUG 24
MongoDB total error percent: 0.08771929824561403

Number of Apache Spark reports: 307
Apache Spark number of bugs: 144
Apache Spark number of nugs: 163
Number of Apache Spark missing labels: 0
Number of Apache Spark misclassified as NUG 4
Number of Apache Spark misclassified as BUG 11
Apache Spark total error percent: 0.048859934853420196

Number of WildFly reports: 151
WildFly number of bugs: 103
WildFly number of nugs: 48
Number of WildFly missing labels: 0
Number of WildFly misclassified as NUG 0
Number of WildFly misclassified as BUG 11
WildFly total error percent: 0.0728476821192053



## 1.3 Combined Data

In [14]:
df_issues_original = df_issues_original.select('ID','CLASSIFIED','TYPE','SUMMARY')
df_original_with_spark = df_issues_original.union(df_spark.select('ID','CLASSIFIED','TYPE','SUMMARY'))
df_original_with_mongodb = df_issues_original.union(df_mongodb.select('ID','CLASSIFIED','TYPE','SUMMARY'))
df_original_with_wildfly = df_issues_original.union(df_wildfly.select('ID','CLASSIFIED','TYPE','SUMMARY'))
 
df_all_combined = df_issues_original.union(df_spark.select('ID','CLASSIFIED','TYPE','SUMMARY')).union(df_mongodb.select('ID','CLASSIFIED','TYPE','SUMMARY')).union(df_wildfly.select('ID','CLASSIFIED','TYPE','SUMMARY'))

# 2. Data Labeling Quality Measurement

## 2.1 Fleiss Kappa

In [15]:
file_location = "https://raw.githubusercontent.com/hasnil/bug-or-nug/main/Data%20As%20CSVs/MemberRatings.csv"
df_csv = pandas.read_csv(file_location)
temp_file_location = '/content/drive/My Drive/Colab Notebooks/MemberRatings.csv'
df_csv.to_csv(temp_file_location)

df_ratings = spark.read.option("header", True).csv(temp_file_location).cache()
df_ratings = df_ratings.drop("_c0")

In [16]:
# Fleiss kappa, an adaptation of cohen kappa is used 
pandas_df = df_ratings.toPandas()
pandas_df = pandas_df.replace('BUG',1, regex=True)
pandas_df = pandas_df.replace('NUG',0, regex=True)
 
temp = pandas.DataFrame(0, index=numpy.arange(len(pandas_df)), columns=[0,1])
 
for i in range(len(pandas_df)):
  for j in range(3):
    if(pandas_df.iloc[i][j] == 0):
      temp.iloc[i][0] += 1
    if(pandas_df.iloc[i][j] == 1):
      temp.iloc[i][1] += 1
 
kappa_df = temp
 
number_of_raters = 3
number_of_categories = 2
number_of_subjects = len(kappa_df)
sum_of_all_cells = kappa_df.to_numpy().sum()
small_p = kappa_df.sum(axis=0)/sum_of_all_cells
big_p = 1/(number_of_raters*(number_of_raters - 1))*((kappa_df**2).sum(axis=1) - number_of_raters)
big_p_sum = big_p.sum()
term_1 = big_p_sum/number_of_subjects
term_2 = (small_p**2).sum()
fleiss_kappa = (term_1 - term_2)/(1 - term_2)
print('The value for Fleiss Kappa is', fleiss_kappa)


The value for Fleiss Kappa is 0.9549954995499551


## 2.2 Percentage Agreement

In [17]:
# File already modified to have 1 for bug and 0 for nug
file_location = "https://raw.githubusercontent.com/hasnil/bug-or-nug/main/Data%20As%20CSVs/BinaryMemberRatings.csv"
df_csv = pandas.read_csv(file_location)
temp_file_location = '/content/drive/My Drive/Colab Notebooks/BinaryMemberRatings.csv'
df_csv.to_csv(temp_file_location)

df_ratings_bin = spark.read.option("header", True).csv(temp_file_location)
df_ratings_bin = df_ratings_bin.drop("_c0")

In [18]:
# Percentage agreement# is calculated in this way
pandas_df = df_ratings_bin.toPandas().astype(int)
pandas_df['Diff b/w Alan and Aaron'] = pandas_df['Alan Ratings'] - pandas_df['Aaron Ratings']
pandas_df['Diff b/w Alan and Lotfi'] = pandas_df['Lotfi Ratings'] - pandas_df['Alan Ratings']
pandas_df['Diff b/w Aaron and Lotfi'] = pandas_df['Lotfi Ratings'] - pandas_df['Aaron Ratings']
count_first = len(pandas_df['Diff b/w Alan and Aaron'][pandas_df['Diff b/w Alan and Aaron'] == 0])
count_second = len(pandas_df['Diff b/w Alan and Lotfi'][pandas_df['Diff b/w Alan and Lotfi'] == 0])
count_third = len(pandas_df['Diff b/w Aaron and Lotfi'][pandas_df['Diff b/w Alan and Lotfi'] == 0])
total_ratings = len(pandas_df)
percentage_agreement = 100*(count_first/total_ratings + count_second/total_ratings + count_third/total_ratings)/3
print('The overall percentage agreement is', percentage_agreement)

The overall percentage agreement is 97.875


In [19]:
alan_aaron_agreement = df_ratings_bin.filter(col('Alan Ratings') == col('Aaron Ratings')).count()/df_ratings.count()
alan_lotfi_agreement = df_ratings_bin.filter(col('Alan Ratings') == col('Lotfi Ratings')).count()/df_ratings.count()
aaron_lotfi_agreement = df_ratings_bin.filter(col('Aaron Ratings') == col('Lotfi Ratings')).count()/df_ratings.count()
 
print('The percentage agreement between Alan and Aaron is', alan_aaron_agreement*100, "%")
print('The percentage agreement between Alan and Lotfi is', alan_lotfi_agreement*100, "%")
print('The percentage agreement between Aaron and Lotfi is', aaron_lotfi_agreement*100, "%")

The percentage agreement between Alan and Aaron is 97.375 %
The percentage agreement between Alan and Lotfi is 98.125 %
The percentage agreement between Aaron and Lotfi is 97.75 %


# 3. Data Preprocessing

## 3.1 Text Preprocessing Functions

In [20]:
def extractHTML(i, columnName, columnsList): 
    """ 
    Used to extract words from textual contents with html tags
    """
    placement = columnsList.index(columnName)
    
    row = i.asDict()
    soup = BeautifulSoup(row[columnName])
    
    for tag in soup.find_all('code'):
      tag.decompose()
 
    for tag in soup.find_all('pre'):
      tag.decompose()
 
    for tag in soup.find_all('a'):
      tag.decompose()
 
    t = soup.get_text().split()
 
    for i in range(len(t)):
      t[i] = contractions.fix(t[i])
 
    t = " ".join(t)
    
    return_list = []
    
    for cn in columnsList:
      if cn == columnName:
        return_list.append(t)
      else:
        return_list.append(row[cn])
    return return_list
   
def remove_noise(df_column):
    """
    This function may be used to clean more noise from textual contents
    """
    df_column = lower(df_column) # Makes text lowercase
    df_column = regexp_replace(df_column, "(https?\://)\S+", " ") # Removes remaining links
    df_column = regexp_replace(df_column, "[^a-zA-Z0-9\\s]", " ") # Keeps only alphanumerics
    df_column = regexp_replace(df_column, "[\d-]", " ") # Removes digits
    df_column = regexp_replace(df_column,'\\p{Punct}'," ") # Removes punctuation
    
    return trim(df_column)
  
#Modified list of stop words used for removal
list_of_stop_words = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", 
 "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", 
 "them", "their", "theirs", "themselves", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", 
 "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "i'm", "you're", "he's", "she's", 
 "it's", "we're", "they're", "i've", "you've", "we've", "they've", "i'd", "you'd", "he'd", "she'd", "we'd", "they'd", "i'll", "you'll", 
 "he'll", "she'll", "we'll", "they'll", "let's", "that's", "who's", "what's", "here's", "there's", "when's", "where's", "why's", "how's", "a", 
 "an", "the", "and", "or", "because", "as", "of", "at", "by", "for", "with", "about", "against", "between", "into", 
 "through", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "further", "once", 
 "here", "there", "all", "both", "each", "few", "more", "other", "such", "only", "own", "so", "than", "too", "very"]
  
def tokenize_and_remove_stopwords(df, *cols):
  """
  This function may be used to tokenize and remove stopwords from textual contents
  """
  for i in cols:
    output_col_name = "Tokenized " + i
    regex_tokenizer = RegexTokenizer(inputCol = i, outputCol = output_col_name, pattern="\\W+")
    df = regex_tokenizer.transform(df)
    remover = StopWordsRemover().setStopWords(list_of_stop_words).setInputCol(output_col_name)
    remover.setOutputCol(output_col_name + " with Stop Words Removed")
    df = remover.transform(df)
  return df
 
 
def get_top_terms(df_issues_stem_string, MAX_TERMS = 100):
  """
  Used to limit the number of terms for models to MAX_TERMS. If value entered
  as second argument is greater than number of terms, all will be included.
  """
  #Default max number of terms used in DTM is 100
  
  all_words_string = df_issues_stem_string.select('Stemmed String').rdd.flatMap(lambda x: x).collect()
 
  all_words_string = " ".join(all_words_string).split()
 
  d = dict(Counter(all_words_string))
 
  #Remove terms of length = 1 for more meaningful results
  for key in list(d.keys()):
    if len(key) <= 1:
      del d[key]
 
  word_count = sorted(d, key = d.get, reverse = True)
  if(MAX_TERMS == 'all'):
    return word_count
  if(len(word_count) < MAX_TERMS):
    return word_count
  top_terms = word_count[0:MAX_TERMS]
  return top_terms
 
 
def prune_text(row, top_terms):
  """
  This function is used to remove all words that are
  not a part of the list of most frequent words
  """
  row_dictionary = row.asDict()
  new_list = []
  for word in row_dictionary.get('Stemmed Textual Contents'):
    if word in top_terms:
      new_list.append(word)
  row_dictionary['Pruned Textual Contents'] = new_list
  return row_dictionary
 
 
def apply_pruning(df_issues_stem_string, top_terms):
  """
  This function applies the pruning procedure
  """
  headers = df_issues_stem_string.schema
  if('Pruned Textual Contents' in headers.names):
    pass
  else:
    headers = headers.add('Pruned Textual Contents', ArrayType(StringType()))
 
  pruned_rdd = df_issues_stem_string.rdd.map(lambda row: prune_text(row, top_terms))
  df_issues_stem_string = spark.createDataFrame(pruned_rdd, headers)
  return df_issues_stem_string
 
 
def apply_normalizer(df, p_norm):
  """
  Normalizes each vector to have unit norm, with specified p-norm. 2.0 is Euclidean norm
  """
  if(p_norm == 'none'):
    return df
  
  normalizer = Normalizer(inputCol='features', outputCol='Normalized Features', p=p_norm)
  df_normalized = normalizer.transform(df)
  return df_normalized

## 3.2 Combined Preprocessing Steps

In [21]:
def preprocess_text(df_issues, p_norm, MAX_TERMS = 100):
  """
  This function will perform the text preprocessing
  procedure on a dataframe for the bug issue reports
  """
  # The preprocessing procedure follows these steps: 
  # Step 1: Remove extraneous html contents
  headers = df_issues.schema
  issues_rdd = df_issues.rdd
  issues_rdd = issues_rdd.map(lambda x: extractHTML(x, 'SUMMARY', headers.names))
  df_issues_modified = spark.createDataFrame(issues_rdd, headers)
 
  # Step 2: Noise removal  
  df_issues_cleaned = df_issues_modified.withColumn("Cleaned Summary", remove_noise(col("SUMMARY")))
 
  # Step 3: Tokenizing and removing stopwords
  df_issues_tokenized_and_cleaned = tokenize_and_remove_stopwords(df_issues_cleaned, "Cleaned Summary")
 
  # Step 4: Stemming
  stemmer = PorterStemmer()
  stem_udf = udf(lambda tokenized: [stemmer.stem(tokenized) for tokenized in tokenized], ArrayType(StringType()))
 
  df_issues_stemmed = df_issues_tokenized_and_cleaned.withColumn('Stemmed Textual Contents', stem_udf('Tokenized Cleaned Summary with Stop Words Removed'))
 
  # Step 5: Creating string column from stemmed textual contents
  df_issues_stem_string = df_issues_stemmed.withColumn('Stemmed String', concat_ws(" ",col("Stemmed Textual Contents")))
  
  # Step 6: Apply pruning
  df_prune = apply_pruning(df_issues_stem_string, get_top_terms(df_issues_stem_string, MAX_TERMS))
  
  # Step 7: Add CountVectorizer DTM column
  countVectors = CountVectorizer(inputCol="Pruned Textual Contents", outputCol="DTM")  
  cvmodel = countVectors.fit(df_prune)  
  dtm_df = cvmodel.transform(df_prune)
  
  dtm_df = dtm_df.withColumnRenamed('DTM', 'features')
  
  # Step 8: Normalize DTM
  df_normalized = apply_normalizer(dtm_df, p_norm)
  
  # Step 9: Encode label
  indexer = StringIndexer(inputCol = 'CLASSIFIED', outputCol = 'label')
  df_result = indexer.fit(df_normalized).transform(df_normalized)
  
  if(p_norm == 'none'):
    return df_result.select('ID','CLASSIFIED','TYPE','SUMMARY', 'Pruned Textual Contents', 'features', 'label')
  else:
    return df_result.select('ID','CLASSIFIED','TYPE','SUMMARY', 'Pruned Textual Contents', 'features', 'Normalized Features', 'label')

In [22]:
#@title Previously used for normalizing raw term frequencies in DTM
"""
#Previously used for normalizing raw term frequencies in DTM

def apply_transform_2(row):
  row_dictionary = row.asDict()
  original_dtm = row_dictionary.get("DTM")
  total = 0
  for val in original_dtm:
    total += val*val
  if(total != 0):
    denominator = math.sqrt(total)
    for i in range(len(original_dtm)):
      original_dtm[i] = original_dtm[i]/denominator
  row_dictionary['DTM'] = original_dtm
  return row_dictionary
  
def apply_transform_3(row):
  row_dictionary = row.asDict()
  original_dtm = row_dictionary.get("DTM")
  for i in range(len(original_dtm)):
    if(original_dtm[i] != 0):
      original_dtm[i] = 1 + math.log(original_dtm[i])      
  row_dictionary['DTM'] = original_dtm
  return row_dictionary
  
def apply_transform_4(row):
  row_dictionary = row.asDict()
  original_dtm = row_dictionary.get("DTM")
  # W2
  for i in range(len(original_dtm)):
    if(original_dtm[i] != 0):
      original_dtm[i] = 1 + math.log(original_dtm[i])
  # L2
  total = 0
  for val in original_dtm:
    total += val*val
  if(total != 0):
    denominator = math.sqrt(total)
    for i in range(len(original_dtm)):
      original_dtm[i] = original_dtm[i]/denominator
  row_dictionary['DTM'] = original_dtm
  return row_dictionary
  
def apply_transform_5(row):
  # alpha assumed 0
  row_dictionary = row.asDict()
  original_dtm = row_dictionary.get("DTM")
  max_val = max(original_dtm)
  if(max_val != 0):
    for i in range(len(original_dtm)):
      original_dtm[i] = original_dtm[i]/max_val
  row_dictionary['DTM'] = original_dtm
  return row_dictionary

def apply_transform_6(row):
  # alpha assumed 0
  # W3
  row_dictionary = row.asDict()
  original_dtm = row_dictionary.get("DTM")
  max_val = max(original_dtm)
  if(max_val != 0):
    for i in range(len(original_dtm)):
      original_dtm[i] = original_dtm[i]/max_val
  # L2
  total = 0
  for val in original_dtm:
    total += val*val
  if(total != 0):
    denominator = math.sqrt(total)
    for i in range(len(original_dtm)):
      original_dtm[i] = original_dtm[i]/denominator
  row_dictionary['DTM'] = original_dtm
  return row_dictionary
  
def apply_transform_7(row):
  row_dictionary = row.asDict()
  original_dtm = row_dictionary.get("DTM")
  total = sum(original_dtm)
  if(total != 0):
    denominator = total
    for i in range(len(original_dtm)):
      original_dtm[i] = original_dtm[i]/denominator
  row_dictionary['DTM'] = original_dtm
  return row_dictionary


def apply_normalization(rdd, number):
  #This function will apply the chosen number's
  #normalization to the document term matrix
  
  if(number == 1):
    pass #Uses raw term frequency
  elif(number == 2):
    rdd = rdd.map(apply_transform_2)
  elif(number == 3):
    rdd = rdd.map(apply_transform_3)
  elif(number == 4):
    rdd = rdd.map(apply_transform_4)
  elif(number == 5):
    rdd = rdd.map(apply_transform_5)
  elif(number == 6):
    rdd = rdd.map(apply_transform_6)
  elif(number == 7):
    rdd = rdd.map(apply_transform_7)
  return rdd
  
"""

'\n#Previously used for normalizing raw term frequencies in DTM\n\ndef apply_transform_2(row):\n  row_dictionary = row.asDict()\n  original_dtm = row_dictionary.get("DTM")\n  total = 0\n  for val in original_dtm:\n    total += val*val\n  if(total != 0):\n    denominator = math.sqrt(total)\n    for i in range(len(original_dtm)):\n      original_dtm[i] = original_dtm[i]/denominator\n  row_dictionary[\'DTM\'] = original_dtm\n  return row_dictionary\n  \ndef apply_transform_3(row):\n  row_dictionary = row.asDict()\n  original_dtm = row_dictionary.get("DTM")\n  for i in range(len(original_dtm)):\n    if(original_dtm[i] != 0):\n      original_dtm[i] = 1 + math.log(original_dtm[i])      \n  row_dictionary[\'DTM\'] = original_dtm\n  return row_dictionary\n  \ndef apply_transform_4(row):\n  row_dictionary = row.asDict()\n  original_dtm = row_dictionary.get("DTM")\n  # W2\n  for i in range(len(original_dtm)):\n    if(original_dtm[i] != 0):\n      original_dtm[i] = 1 + math.log(original_dtm[i])\n

## 3.3 Splitting Data into Training and Test Sets

In [23]:
# All new plus old - without normalization
df_combined_processed = preprocess_text(df_all_combined, 'none', 100)
df_train, df_test = df_combined_processed.randomSplit([0.8, 0.2], seed = 50)
 
# Original data only - without normalization
df_original_processed = preprocess_text(df_issues_original, 'none', 100)
df_train_original_data, df_test_original_data = df_original_processed.randomSplit([0.8, 0.2], seed = 51)
 
# New data only (all 3 sources) - without normalization
df_new_processed = preprocess_text(df_new_issues, 'none', 100)
df_train_all_new, df_test_all_new = df_new_processed.randomSplit([0.8, 0.2], seed = 52)
 
# Original data with new Spark records - without normalization
df_original_with_spark_processed = preprocess_text(df_original_with_spark, 'none', 100)
df_train_original_with_spark, df_test_with_spark = df_original_with_spark_processed.randomSplit([0.8, 0.2], seed = 53)
 
# Original data with new MongoDB records - without normalization
df_original_with_mongodb_processed = preprocess_text(df_original_with_mongodb, 'none', 100)
df_train_original_with_mongodb, df_test_with_mongodb = df_original_with_mongodb_processed.randomSplit([0.8, 0.2], seed = 54)
 
# Original data with new WildFly recprds  - without normalization
df_original_with_wildfly_processed = preprocess_text(df_original_with_wildfly, 'none', 100)
df_train_original_with_wildfly, df_test_with_wildfly = df_original_with_wildfly_processed.randomSplit([0.8, 0.2], seed = 55)
 
# With normalization ------------------------------------------------------------------------------------------------------------
 
# All new plus old - with normalization
df_combined_processed = preprocess_text(df_all_combined, 2.0, 100)
normalized_df_train, normalized_df_test = df_combined_processed.randomSplit([0.8, 0.2], seed = 56)
 
# Original data only - with normalization
df_original_processed = preprocess_text(df_issues_original, 2.0, 100)
normalized_df_train_original_data, normalized_df_test_original_data = df_original_processed.randomSplit([0.8, 0.2], seed = 56)
 
# New data only (all 3 sources) - with normalization
df_new_processed = preprocess_text(df_new_issues, 2.0, 100)
normalized_df_train_all_new, normalized_df_test_all_new = df_new_processed.randomSplit([0.8, 0.2], seed = 57)
 
# Original data with new Spark records - with normalization
df_original_with_spark_processed = preprocess_text(df_original_with_spark, 2.0, 100)
normalized_df_train_original_with_spark, normalized_df_test_with_spark = df_original_with_spark_processed.randomSplit([0.8, 0.2], seed = 58)
 
# Original data with new MongoDB records - with normalization
df_original_with_mongodb_processed = preprocess_text(df_original_with_mongodb, 2.0, 100)
normalized_df_train_original_with_mongodb, normalized_df_test_with_mongodb = df_original_with_mongodb_processed.randomSplit([0.8, 0.2], seed = 59)
 
# Original data with new WildFly recprds  - with normalization
df_original_with_wildfly_processed = preprocess_text(df_original_with_wildfly, 2.0, 100)
normalized_df_train_original_with_wildfly, normalized_df_test_with_wildfly = df_original_with_wildfly_processed.randomSplit([0.8, 0.2], seed = 60)

# 4. Model Performance on Data (0 is NUG, 1 is BUG)

Models are run with default parameters to compare performance before tuning hyperparameters.

In [24]:
def print_metrics(predictions):
  evaluator = BinaryClassificationEvaluator(labelCol = 'label', rawPredictionCol="rawPrediction")
  area_under_curve = evaluator.evaluate(predictions, {evaluator.metricName: 'areaUnderROC'})
  predictions.groupBy('label', 'prediction').count().show()
  TP = predictions.filter('prediction = 1 AND label = prediction').count()
  TN = predictions.filter('prediction = 0 AND label = prediction').count()
  FP = predictions.filter('prediction = 1 AND label != prediction').count()
  FN = predictions.filter('prediction = 0 AND label != prediction').count()
  accuracy = (TN + TP)/(TN + TP + FN + FP)
  precision = TP/(TP + FP)
  recall = TP/(TP + FN)
  nug_precision = TN/(TN + FN)
  nug_recall = TN/(TN + FP)
  f_nug = 2 * (nug_precision*nug_recall) / (nug_precision + nug_recall)
  f =  2 * (precision * recall) / (precision + recall)
  weighted_f_measure = (f*(TP + FN) + f_nug*(TN + FP))/(TP + TN + FN + FP)
  print('AUC: %0.3f' % area_under_curve)
  print('Accuracy: %0.3f' % accuracy)
  print('Precision: %0.3f' % precision)
  print('Recall: %0.3f' % recall)
  print('F1 score: %0.3f' % f)
  print("Weighted F-Measure: %0.3f" % weighted_f_measure)
  print()

## 4.1 Without Normalization

### 4.1.1 Naive Bayes Classifier Original Data

In [25]:
nb = NaiveBayes(labelCol = 'label', featuresCol = 'features')
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
# Default of 10 folds for cross-validation from paper, 3 used for time saving purposes
NUM_CV_FOLDS = 3
paramGrid = ParamGridBuilder().build()
cv = StratifiedCrossValidator(estimator = nb, estimatorParamMaps = paramGrid, evaluator = evaluator, numFolds = NUM_CV_FOLDS)
cross_val_model= cv.fit(df_train_original_data)
predictions = cross_val_model.transform(df_test_original_data)
print_metrics(predictions)
#print(cross_val_model.avgMetrics)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  230|
|  0.0|       1.0|  111|
|  1.0|       0.0|  167|
|  0.0|       0.0|  636|
+-----+----------+-----+

AUC: 0.525
Accuracy: 0.757
Precision: 0.674
Recall: 0.579
F1 score: 0.623
Weighted F-Measure: 0.752



### 4.1.2 Naive Bayes Classifier Original Plus Spark Data

In [26]:
cross_val_model= cv.fit(df_train_original_with_spark)
predictions = cross_val_model.transform(df_test_with_spark)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  239|
|  0.0|       1.0|  106|
|  1.0|       0.0|  160|
|  0.0|       0.0|  611|
+-----+----------+-----+

AUC: 0.541
Accuracy: 0.762
Precision: 0.693
Recall: 0.599
F1 score: 0.642
Weighted F-Measure: 0.757



### 4.1.3 Naive Bayes Classifier Original Plus MongoDB Data

In [27]:
cross_val_model= cv.fit(df_train_original_with_mongodb)
predictions = cross_val_model.transform(df_test_with_mongodb)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  240|
|  0.0|       1.0|  113|
|  1.0|       0.0|  209|
|  0.0|       0.0|  674|
+-----+----------+-----+

AUC: 0.527
Accuracy: 0.739
Precision: 0.680
Recall: 0.535
F1 score: 0.599
Weighted F-Measure: 0.731



### 4.1.4 Naive Bayes Classifier Original Plus WildFly Data

In [28]:
cross_val_model= cv.fit(df_train_original_with_wildfly)
predictions = cross_val_model.transform(df_test_with_wildfly)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  232|
|  0.0|       1.0|   96|
|  1.0|       0.0|  199|
|  0.0|       0.0|  669|
+-----+----------+-----+

AUC: 0.559
Accuracy: 0.753
Precision: 0.707
Recall: 0.538
F1 score: 0.611
Weighted F-Measure: 0.744



### 4.1.5 Naive Bayes Classifier New Data Only

In [29]:
cross_val_model= cv.fit(df_train_all_new)
predictions = cross_val_model.transform(df_test_all_new)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   60|
|  0.0|       1.0|   18|
|  1.0|       0.0|   36|
|  0.0|       0.0|   66|
+-----+----------+-----+

AUC: 0.569
Accuracy: 0.700
Precision: 0.769
Recall: 0.625
F1 score: 0.690
Weighted F-Measure: 0.699



### 4.1.6 Naive Bayes Classifier Original Plus All New Data

In [30]:
cross_val_model= cv.fit(df_train)
predictions = cross_val_model.transform(df_test)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  249|
|  0.0|       1.0|  119|
|  1.0|       0.0|  211|
|  0.0|       0.0|  678|
+-----+----------+-----+

AUC: 0.550
Accuracy: 0.737
Precision: 0.677
Recall: 0.541
F1 score: 0.601
Weighted F-Measure: 0.730



### 4.1.7 Linear SVC Classifier Original Data

In [31]:
lsvc = LinearSVC(labelCol = 'label', featuresCol = 'features')
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
# Default of 10 folds for cross-validation from paper, 3 used for time saving purposes
NUM_CV_FOLDS = 3
paramGrid = ParamGridBuilder().build()
cv = StratifiedCrossValidator(estimator = lsvc, estimatorParamMaps = paramGrid, evaluator = evaluator, numFolds = NUM_CV_FOLDS)
cross_val_model= cv.fit(df_train_original_data)
predictions = cross_val_model.transform(df_test_original_data)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  216|
|  0.0|       1.0|   84|
|  1.0|       0.0|  181|
|  0.0|       0.0|  663|
+-----+----------+-----+

AUC: 0.809
Accuracy: 0.768
Precision: 0.720
Recall: 0.544
F1 score: 0.620
Weighted F-Measure: 0.759



### 4.1.8 Linear SVC Classifier Original Plus Spark Data

In [32]:
cross_val_model= cv.fit(df_train_original_with_spark)
predictions = cross_val_model.transform(df_test_with_spark)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  216|
|  0.0|       1.0|   74|
|  1.0|       0.0|  183|
|  0.0|       0.0|  643|
+-----+----------+-----+

AUC: 0.827
Accuracy: 0.770
Precision: 0.745
Recall: 0.541
F1 score: 0.627
Weighted F-Measure: 0.760



### 4.1.9 Linear SVC Classifier Original Plus MongoDB Data

In [33]:
cross_val_model= cv.fit(df_train_original_with_mongodb)
predictions = cross_val_model.transform(df_test_with_mongodb)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  239|
|  0.0|       1.0|  102|
|  1.0|       0.0|  210|
|  0.0|       0.0|  685|
+-----+----------+-----+

AUC: 0.801
Accuracy: 0.748
Precision: 0.701
Recall: 0.532
F1 score: 0.605
Weighted F-Measure: 0.738



### 4.1.10 Linear SVC Classifier Original Plus WildFly Data

In [34]:
cross_val_model= cv.fit(df_train_original_with_wildfly)
predictions = cross_val_model.transform(df_test_with_wildfly)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  221|
|  0.0|       1.0|   75|
|  1.0|       0.0|  210|
|  0.0|       0.0|  690|
+-----+----------+-----+

AUC: 0.826
Accuracy: 0.762
Precision: 0.747
Recall: 0.513
F1 score: 0.608
Weighted F-Measure: 0.749



### 4.1.11 Linear SVC Classifier New Data Only

In [35]:
cross_val_model= cv.fit(df_train_all_new)
predictions = cross_val_model.transform(df_test_all_new)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   75|
|  0.0|       1.0|   30|
|  1.0|       0.0|   21|
|  0.0|       0.0|   54|
+-----+----------+-----+

AUC: 0.764
Accuracy: 0.717
Precision: 0.714
Recall: 0.781
F1 score: 0.746
Weighted F-Measure: 0.715



### 4.1.12 Linear SVC Classifier Original Plus All New Data

In [36]:
cross_val_model= cv.fit(df_train)
predictions = cross_val_model.transform(df_test)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  232|
|  0.0|       1.0|   90|
|  1.0|       0.0|  228|
|  0.0|       0.0|  707|
+-----+----------+-----+

AUC: 0.805
Accuracy: 0.747
Precision: 0.720
Recall: 0.504
F1 score: 0.593
Weighted F-Measure: 0.735



### 4.1.13 Random Forest Classifier Original Data

In [37]:
rf = RandomForestClassifier(labelCol = 'label', featuresCol = 'features')
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
# Default of 10 folds for cross-validation from paper, 3 used for time saving purposes
NUM_CV_FOLDS = 3
paramGrid = ParamGridBuilder().build()
cv = StratifiedCrossValidator(estimator = rf, estimatorParamMaps = paramGrid, evaluator = evaluator, numFolds = NUM_CV_FOLDS)
cross_val_model= cv.fit(df_train_original_data)
predictions = cross_val_model.transform(df_test_original_data)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   40|
|  0.0|       1.0|    9|
|  1.0|       0.0|  357|
|  0.0|       0.0|  738|
+-----+----------+-----+

AUC: 0.782
Accuracy: 0.680
Precision: 0.816
Recall: 0.101
F1 score: 0.179
Weighted F-Measure: 0.585



### 4.1.14 Random Forest Classifier Original Plus Spark Data

In [38]:
cross_val_model= cv.fit(df_train_original_with_spark)
predictions = cross_val_model.transform(df_test_with_spark)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  116|
|  0.0|       1.0|   34|
|  1.0|       0.0|  283|
|  0.0|       0.0|  683|
+-----+----------+-----+

AUC: 0.776
Accuracy: 0.716
Precision: 0.773
Recall: 0.291
F1 score: 0.423
Weighted F-Measure: 0.673



### 4.1.15 Random Forest Classifier Original Plus MongoDB Data

In [39]:
cross_val_model= cv.fit(df_train_original_with_mongodb)
predictions = cross_val_model.transform(df_test_with_mongodb)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   48|
|  0.0|       1.0|    8|
|  1.0|       0.0|  401|
|  0.0|       0.0|  779|
+-----+----------+-----+

AUC: 0.752
Accuracy: 0.669
Precision: 0.857
Recall: 0.107
F1 score: 0.190
Weighted F-Measure: 0.573



### 4.1.16 Random Forest Classifier Original Plus WildFly Data

In [40]:
cross_val_model= cv.fit(df_train_original_with_wildfly)
predictions = cross_val_model.transform(df_test_with_wildfly)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   61|
|  0.0|       1.0|   10|
|  1.0|       0.0|  370|
|  0.0|       0.0|  755|
+-----+----------+-----+

AUC: 0.793
Accuracy: 0.682
Precision: 0.859
Recall: 0.142
F1 score: 0.243
Weighted F-Measure: 0.599



### 4.1.17 Random Forest Classifier New Data Only

In [41]:
cross_val_model= cv.fit(df_train_all_new)
predictions = cross_val_model.transform(df_test_all_new)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   40|
|  0.0|       1.0|    7|
|  1.0|       0.0|   56|
|  0.0|       0.0|   77|
+-----+----------+-----+

AUC: 0.780
Accuracy: 0.650
Precision: 0.851
Recall: 0.417
F1 score: 0.559
Weighted F-Measure: 0.630



### 4.1.18 Random Forest Classifier Original Plus All New Data

In [42]:
cross_val_model= cv.fit(df_train)
predictions = cross_val_model.transform(df_test)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  169|
|  0.0|       1.0|   60|
|  1.0|       0.0|  291|
|  0.0|       0.0|  737|
+-----+----------+-----+

AUC: 0.761
Accuracy: 0.721
Precision: 0.738
Recall: 0.367
F1 score: 0.491
Weighted F-Measure: 0.692



## 4.2 With Normalization p = 2.0

### 4.2.1 Naive Bayes Classifier Original Data

In [43]:
nb = NaiveBayes(labelCol = 'label', featuresCol = 'Normalized Features')
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
NUM_CV_FOLDS = 3
paramGrid = ParamGridBuilder().build()
cv = StratifiedCrossValidator(estimator = nb, estimatorParamMaps = paramGrid, evaluator = evaluator, numFolds = NUM_CV_FOLDS)
cross_val_model= cv.fit(normalized_df_train_original_data)
predictions = cross_val_model.transform(normalized_df_test_original_data)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  188|
|  0.0|       1.0|   62|
|  1.0|       0.0|  180|
|  0.0|       0.0|  636|
+-----+----------+-----+

AUC: 0.579
Accuracy: 0.773
Precision: 0.752
Recall: 0.511
F1 score: 0.608
Weighted F-Measure: 0.760



### 4.2.2 Naive Bayes Classifier Original Plus Spark Data

In [44]:
cross_val_model= cv.fit(normalized_df_train_original_with_spark)
predictions = cross_val_model.transform(normalized_df_test_with_spark)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  218|
|  0.0|       1.0|   79|
|  1.0|       0.0|  192|
|  0.0|       0.0|  680|
+-----+----------+-----+

AUC: 0.588
Accuracy: 0.768
Precision: 0.734
Recall: 0.532
F1 score: 0.617
Weighted F-Measure: 0.758



### 4.2.3 Naive Bayes Classifier Original Plus MongoDB Data

In [45]:
cross_val_model= cv.fit(normalized_df_train_original_with_mongodb)
predictions = cross_val_model.transform(normalized_df_test_with_mongodb)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  227|
|  0.0|       1.0|   86|
|  1.0|       0.0|  191|
|  0.0|       0.0|  706|
+-----+----------+-----+

AUC: 0.583
Accuracy: 0.771
Precision: 0.725
Recall: 0.543
F1 score: 0.621
Weighted F-Measure: 0.762



### 4.2.4 Naive Bayes Classifier Original Plus WildFly Data

In [46]:
cross_val_model= cv.fit(normalized_df_train_original_with_wildfly)
predictions = cross_val_model.transform(normalized_df_test_with_wildfly)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  201|
|  0.0|       1.0|   82|
|  1.0|       0.0|  193|
|  0.0|       0.0|  661|
+-----+----------+-----+

AUC: 0.618
Accuracy: 0.758
Precision: 0.710
Recall: 0.510
F1 score: 0.594
Weighted F-Measure: 0.747



### 4.2.5 Naive Bayes Classifier New Data Only

In [47]:
cross_val_model= cv.fit(normalized_df_train_all_new)
predictions = cross_val_model.transform(normalized_df_test_all_new)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   48|
|  0.0|       1.0|   25|
|  1.0|       0.0|   18|
|  0.0|       0.0|   53|
+-----+----------+-----+

AUC: 0.561
Accuracy: 0.701
Precision: 0.658
Recall: 0.727
F1 score: 0.691
Weighted F-Measure: 0.702



### 4.2.6 Naive Bayes Classifier Original Plus All New Data

In [48]:
cross_val_model= cv.fit(normalized_df_train)
predictions = cross_val_model.transform(normalized_df_test)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  213|
|  0.0|       1.0|   88|
|  1.0|       0.0|  226|
|  0.0|       0.0|  687|
+-----+----------+-----+

AUC: 0.579
Accuracy: 0.741
Precision: 0.708
Recall: 0.485
F1 score: 0.576
Weighted F-Measure: 0.728



### 4.2.7 Linear SVC Classifier Original Data

In [49]:
lsvc = LinearSVC(labelCol = 'label', featuresCol = 'Normalized Features')
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
NUM_CV_FOLDS = 3
paramGrid = ParamGridBuilder().build()
cv = StratifiedCrossValidator(estimator = lsvc, estimatorParamMaps = paramGrid, evaluator = evaluator, numFolds = NUM_CV_FOLDS)
cross_val_model= cv.fit(normalized_df_train_original_data)
predictions = cross_val_model.transform(normalized_df_test_original_data)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  201|
|  0.0|       1.0|   82|
|  1.0|       0.0|  167|
|  0.0|       0.0|  616|
+-----+----------+-----+

AUC: 0.831
Accuracy: 0.766
Precision: 0.710
Recall: 0.546
F1 score: 0.618
Weighted F-Measure: 0.758



### 4.2.8 Linear SVC Classifier Original Plus Spark Data

In [50]:
cross_val_model= cv.fit(normalized_df_train_original_with_spark)
predictions = cross_val_model.transform(normalized_df_test_with_spark)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  224|
|  0.0|       1.0|   87|
|  1.0|       0.0|  186|
|  0.0|       0.0|  672|
+-----+----------+-----+

AUC: 0.816
Accuracy: 0.766
Precision: 0.720
Recall: 0.546
F1 score: 0.621
Weighted F-Measure: 0.758



### 4.2.9 Linear SVC Classifier Original Plus MongoDB Data

In [51]:
cross_val_model= cv.fit(normalized_df_train_original_with_mongodb)
predictions = cross_val_model.transform(normalized_df_test_with_mongodb)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  250|
|  0.0|       1.0|   99|
|  1.0|       0.0|  168|
|  0.0|       0.0|  693|
+-----+----------+-----+

AUC: 0.837
Accuracy: 0.779
Precision: 0.716
Recall: 0.598
F1 score: 0.652
Weighted F-Measure: 0.774



### 4.2.10 Linear SVC Classifier Original Plus Wildfly Data

In [52]:
cross_val_model= cv.fit(normalized_df_train_original_with_wildfly)
predictions = cross_val_model.transform(normalized_df_test_with_wildfly)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  219|
|  0.0|       1.0|  110|
|  1.0|       0.0|  175|
|  0.0|       0.0|  633|
+-----+----------+-----+

AUC: 0.802
Accuracy: 0.749
Precision: 0.666
Recall: 0.556
F1 score: 0.606
Weighted F-Measure: 0.743



### 4.2.11 Linear SVC Classifier New Data Only

In [53]:
cross_val_model= cv.fit(normalized_df_train_all_new)
predictions = cross_val_model.transform(normalized_df_test_all_new)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   44|
|  0.0|       1.0|   27|
|  1.0|       0.0|   22|
|  0.0|       0.0|   51|
+-----+----------+-----+

AUC: 0.731
Accuracy: 0.660
Precision: 0.620
Recall: 0.667
F1 score: 0.642
Weighted F-Measure: 0.660



### 4.2.12 Linear SVC Classifier Original Plus All New Data

In [54]:
cross_val_model= cv.fit(normalized_df_train)
predictions = cross_val_model.transform(normalized_df_test)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  238|
|  0.0|       1.0|  104|
|  1.0|       0.0|  201|
|  0.0|       0.0|  671|
+-----+----------+-----+

AUC: 0.806
Accuracy: 0.749
Precision: 0.696
Recall: 0.542
F1 score: 0.609
Weighted F-Measure: 0.741



### 4.2.13 Random Forest Classifier Original Data

In [55]:
rf = RandomForestClassifier(labelCol = 'label', featuresCol = 'Normalized Features')
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
NUM_CV_FOLDS = 3
paramGrid = ParamGridBuilder().build()
cv = StratifiedCrossValidator(estimator = rf, estimatorParamMaps = paramGrid, evaluator = evaluator, numFolds = NUM_CV_FOLDS)
cross_val_model= cv.fit(normalized_df_train_original_data)
predictions = cross_val_model.transform(normalized_df_test_original_data)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   46|
|  0.0|       1.0|   12|
|  1.0|       0.0|  322|
|  0.0|       0.0|  686|
+-----+----------+-----+

AUC: 0.787
Accuracy: 0.687
Precision: 0.793
Recall: 0.125
F1 score: 0.216
Weighted F-Measure: 0.601



### 4.2.14 Random Forest Classifier Original Plus Spark Data

In [56]:
cross_val_model= cv.fit(normalized_df_train_original_with_spark)
predictions = cross_val_model.transform(normalized_df_test_with_spark)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   42|
|  0.0|       1.0|   10|
|  1.0|       0.0|  368|
|  0.0|       0.0|  749|
+-----+----------+-----+

AUC: 0.769
Accuracy: 0.677
Precision: 0.808
Recall: 0.102
F1 score: 0.182
Weighted F-Measure: 0.582



### 4.2.15 Random Forest Classifier Original Plus MongoDB Data

In [57]:
cross_val_model= cv.fit(normalized_df_train_original_with_mongodb)
predictions = cross_val_model.transform(normalized_df_test_with_mongodb)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   55|
|  0.0|       1.0|   11|
|  1.0|       0.0|  363|
|  0.0|       0.0|  781|
+-----+----------+-----+

AUC: 0.788
Accuracy: 0.691
Precision: 0.833
Recall: 0.132
F1 score: 0.227
Weighted F-Measure: 0.607



### 4.2.16 Random Forest Classifier Original Plus Wildfly Data

In [58]:
cross_val_model= cv.fit(normalized_df_train_original_with_wildfly)
predictions = cross_val_model.transform(normalized_df_test_with_wildfly)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   74|
|  0.0|       1.0|   28|
|  1.0|       0.0|  320|
|  0.0|       0.0|  715|
+-----+----------+-----+

AUC: 0.746
Accuracy: 0.694
Precision: 0.725
Recall: 0.188
F1 score: 0.298
Weighted F-Measure: 0.629



### 4.2.17 Random Forest Classifier New Data Only

In [59]:
cross_val_model= cv.fit(normalized_df_train_all_new)
predictions = cross_val_model.transform(normalized_df_test_all_new)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   56|
|  0.0|       1.0|   40|
|  1.0|       0.0|   10|
|  0.0|       0.0|   38|
+-----+----------+-----+

AUC: 0.745
Accuracy: 0.653
Precision: 0.583
Recall: 0.848
F1 score: 0.691
Weighted F-Measure: 0.644



### 4.2.18 Random Forest Classifier Original Plus All New Data

In [60]:
cross_val_model= cv.fit(normalized_df_train)
predictions = cross_val_model.transform(normalized_df_test)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  119|
|  0.0|       1.0|   52|
|  1.0|       0.0|  320|
|  0.0|       0.0|  723|
+-----+----------+-----+

AUC: 0.761
Accuracy: 0.694
Precision: 0.696
Recall: 0.271
F1 score: 0.390
Weighted F-Measure: 0.649



# 5. Effect of Different Hyperparameters on Model Performances (F measure) with All Combined Data

## 5.1 Without Normalization (featuresCol = 'features')

### 5.1.1 Naive Bayes Classifier

In [61]:
nb = NaiveBayes(labelCol = 'label', featuresCol = 'features')
paramGrid = ParamGridBuilder().addGrid(nb.smoothing, [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]).build()
 
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
NUM_CV_FOLDS = 2
 
cv = StratifiedCrossValidator(estimator = nb, estimatorParamMaps = paramGrid, evaluator = evaluator, numFolds = NUM_CV_FOLDS)
 
cvModel = cv.fit(normalized_df_train)
 
predictions = cvModel.transform(normalized_df_test)
print_metrics(predictions)
 
# The following is used from: https://stackoverflow.com/questions/51230726/extract-results-from-crossvalidator-with-paramgrid-in-pyspark
params = [{p.name: v for p, v in m.items()} for m in cvModel.getEstimatorParamMaps()]
pandas.DataFrame.from_dict([{cvModel.getEvaluator().getMetricName(): metric, **ps} for ps, metric in zip(params, cvModel.avgMetrics)])

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  237|
|  0.0|       1.0|  107|
|  1.0|       0.0|  202|
|  0.0|       0.0|  668|
+-----+----------+-----+

AUC: 0.540
Accuracy: 0.745
Precision: 0.689
Recall: 0.540
F1 score: 0.605
Weighted F-Measure: 0.737



,f1,smoothing
0,0.733642,0.0
1,0.733909,0.2
2,0.733559,0.4
3,0.733255,0.6
4,0.733427,0.8
5,0.734045,1.0


### 5.1.2 Linear SVC Classifier

In [62]:
lsvc = LinearSVC(labelCol = 'label', featuresCol = 'features')
 
paramGrid = ParamGridBuilder().addGrid(lsvc.regParam,[0.001, 0.01, 0.1, 1.0]).addGrid(lsvc.maxIter, [10, 15, 20]).build()
 
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
NUM_CV_FOLDS = 2
 
cv = StratifiedCrossValidator(estimator = lsvc, estimatorParamMaps = paramGrid, evaluator = evaluator, numFolds = NUM_CV_FOLDS)
 
cvModel = cv.fit(normalized_df_train)
 
predictions = cvModel.transform(normalized_df_test)
print_metrics(predictions)
 
params = [{p.name: v for p, v in m.items()} for m in cvModel.getEstimatorParamMaps()]
pandas.DataFrame.from_dict([{cvModel.getEvaluator().getMetricName(): metric, **ps} for ps, metric in zip(params, cvModel.avgMetrics)])

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  224|
|  0.0|       1.0|   92|
|  1.0|       0.0|  215|
|  0.0|       0.0|  683|
+-----+----------+-----+

AUC: 0.808
Accuracy: 0.747
Precision: 0.709
Recall: 0.510
F1 score: 0.593
Weighted F-Measure: 0.736



,f1,regParam,maxIter
0,0.734044,0.001,10
1,0.734893,0.001,15
2,0.736044,0.001,20
3,0.731277,0.010,10
4,0.731146,0.010,15
5,0.732746,0.010,20
6,0.723667,0.100,10
7,0.723186,0.100,15
8,0.722645,0.100,20
9,0.549777,1.000,10


### 5.1.3 Random Forest Classifier

In [63]:
rf = RandomForestClassifier(labelCol = 'label', featuresCol = 'features')
 
paramGrid = ParamGridBuilder().addGrid(rf.numTrees,[40, 60, 80]).addGrid(rf.maxDepth, [15, 20, 25, 30]).build()
 
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
NUM_CV_FOLDS = 2
 
cv = StratifiedCrossValidator(estimator = rf, estimatorParamMaps = paramGrid, evaluator = evaluator, numFolds = NUM_CV_FOLDS)
 
cvModel = cv.fit(normalized_df_train)
 
predictions = cvModel.transform(normalized_df_test)
print_metrics(predictions)
 
params = [{p.name: v for p, v in m.items()} for m in cvModel.getEstimatorParamMaps()]
pandas.DataFrame.from_dict([{cvModel.getEvaluator().getMetricName(): metric, **ps} for ps, metric in zip(params, cvModel.avgMetrics)])

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  224|
|  0.0|       1.0|   82|
|  1.0|       0.0|  215|
|  0.0|       0.0|  693|
+-----+----------+-----+

AUC: 0.830
Accuracy: 0.755
Precision: 0.732
Recall: 0.510
F1 score: 0.601
Weighted F-Measure: 0.743



,f1,numTrees,maxDepth
0,0.716213,40,15
1,0.723997,40,20
2,0.723331,40,25
3,0.729318,40,30
4,0.721539,60,15
5,0.724533,60,20
6,0.726005,60,25
7,0.729824,60,30
8,0.715816,80,15
9,0.723239,80,20


In [64]:
trainData, testData = normalized_df_train.randomSplit([0.8, 0.2], seed = 76)
 
numTreesList = [20, 40, 60, 80]
maxDepthList = [15, 20, 25, 30]
maxBinsList = [20, 40, 60]
 
for numTrees in numTreesList:
  for maxDepth in maxDepthList:
    for maxBins in maxBinsList:
      rf = RandomForestClassifier(labelCol = 'label', featuresCol = 'features', numTrees = numTrees, maxDepth = maxDepth, maxBins = maxBins)
      model= rf.fit(trainData)
      predictions = model.transform(testData)
      print('For numTrees:', numTrees, ', maxDepth:', maxDepth, ', maxBins:', maxBins)
      print_metrics(predictions)
      print('---------------------------------------------')

For numTrees: 20 , maxDepth: 15 , maxBins: 20
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  181|
|  0.0|       1.0|   45|
|  1.0|       0.0|  219|
|  0.0|       0.0|  606|
+-----+----------+-----+

AUC: 0.807
Accuracy: 0.749
Precision: 0.801
Recall: 0.453
F1 score: 0.578
Weighted F-Measure: 0.729

---------------------------------------------
For numTrees: 20 , maxDepth: 15 , maxBins: 40
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  181|
|  0.0|       1.0|   45|
|  1.0|       0.0|  219|
|  0.0|       0.0|  606|
+-----+----------+-----+

AUC: 0.807
Accuracy: 0.749
Precision: 0.801
Recall: 0.453
F1 score: 0.578
Weighted F-Measure: 0.729

---------------------------------------------
For numTrees: 20 , maxDepth: 15 , maxBins: 60
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  181|
|  0.0|       1.0|   45|
|  1.0|       0.0|  219|
|  0.0|       0.

The random forest algorithm is found to have the best performance on the test data. MaxBins hyperparameter found to have no effect. Setting numTrees = 20, maxDepth = 30 found to have best results.

## 5.2 With Normalization (featuresCol = 'Normalized Features')

### 5.2.1 Naive Bayes Classifier

In [65]:
nb = NaiveBayes(labelCol = 'label', featuresCol = 'Normalized Features')
paramGrid = ParamGridBuilder().addGrid(nb.smoothing, [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]).build()
 
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
NUM_CV_FOLDS = 2
 
cv = StratifiedCrossValidator(estimator = nb, estimatorParamMaps = paramGrid, evaluator = evaluator, numFolds = NUM_CV_FOLDS)
 
cvModel = cv.fit(normalized_df_train)
 
predictions = cvModel.transform(normalized_df_test)
print_metrics(predictions)
 
params = [{p.name: v for p, v in m.items()} for m in cvModel.getEstimatorParamMaps()]
pandas.DataFrame.from_dict([{cvModel.getEvaluator().getMetricName(): metric, **ps} for ps, metric in zip(params, cvModel.avgMetrics)])

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  216|
|  0.0|       1.0|   90|
|  1.0|       0.0|  223|
|  0.0|       0.0|  685|
+-----+----------+-----+

AUC: 0.585
Accuracy: 0.742
Precision: 0.706
Recall: 0.492
F1 score: 0.580
Weighted F-Measure: 0.729



,f1,smoothing
0,0.732478,0.0
1,0.732245,0.2
2,0.731842,0.4
3,0.731327,0.6
4,0.732305,0.8
5,0.732017,1.0


### 5.2.2 Linear SVC Classifier

In [66]:
lsvc = LinearSVC(labelCol = 'label', featuresCol = 'Normalized Features')
 
paramGrid = ParamGridBuilder().addGrid(lsvc.regParam,[0.001, 0.01, 0.1, 1.0]).addGrid(lsvc.maxIter, [10, 15, 20]).build()
 
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
NUM_CV_FOLDS = 2
 
cv = StratifiedCrossValidator(estimator = lsvc, estimatorParamMaps = paramGrid, evaluator = evaluator, numFolds = NUM_CV_FOLDS)
 
cvModel = cv.fit(normalized_df_train)
 
predictions = cvModel.transform(normalized_df_test)
print_metrics(predictions)
 
params = [{p.name: v for p, v in m.items()} for m in cvModel.getEstimatorParamMaps()]
pandas.DataFrame.from_dict([{cvModel.getEvaluator().getMetricName(): metric, **ps} for ps, metric in zip(params, cvModel.avgMetrics)])

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  236|
|  0.0|       1.0|  101|
|  1.0|       0.0|  203|
|  0.0|       0.0|  674|
+-----+----------+-----+

AUC: 0.805
Accuracy: 0.750
Precision: 0.700
Recall: 0.538
F1 score: 0.608
Weighted F-Measure: 0.741



,f1,regParam,maxIter
0,0.738050,0.001,10
1,0.739868,0.001,15
2,0.737722,0.001,20
3,0.737244,0.010,10
4,0.739562,0.010,15
5,0.739421,0.010,20
6,0.733557,0.100,10
7,0.732844,0.100,15
8,0.733433,0.100,20
9,0.562531,1.000,10


### 5.2.3 Random Forest Classifier

In [67]:
rf = RandomForestClassifier(labelCol = 'label', featuresCol = 'Normalized Features')
 
paramGrid = ParamGridBuilder().addGrid(rf.numTrees,[40, 60, 80]).addGrid(rf.maxDepth, [15, 20, 25, 30]).build()
 
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
NUM_CV_FOLDS = 2
 
cv = StratifiedCrossValidator(estimator = rf, estimatorParamMaps = paramGrid, evaluator = evaluator, numFolds = NUM_CV_FOLDS)
 
cvModel = cv.fit(normalized_df_train)
 
predictions = cvModel.transform(normalized_df_test)
print_metrics(predictions)
 
params = [{p.name: v for p, v in m.items()} for m in cvModel.getEstimatorParamMaps()]
pandas.DataFrame.from_dict([{cvModel.getEvaluator().getMetricName(): metric, **ps} for ps, metric in zip(params, cvModel.avgMetrics)])

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  233|
|  0.0|       1.0|   83|
|  1.0|       0.0|  206|
|  0.0|       0.0|  692|
+-----+----------+-----+

AUC: 0.823
Accuracy: 0.762
Precision: 0.737
Recall: 0.531
F1 score: 0.617
Weighted F-Measure: 0.751



,f1,numTrees,maxDepth
0,0.714626,40,15
1,0.717185,40,20
2,0.720351,40,25
3,0.724080,40,30
4,0.710114,60,15
5,0.718072,60,20
6,0.721777,60,25
7,0.724012,60,30
8,0.710075,80,15
9,0.720667,80,20


The Linear Support Vector Classifier algorithm is found to have the best performance on the normalized test data. Setting regParam = 0.001 and maxIter = 15 found to have best results.

# 6. Distribution of Misclassifications

Retraining best performing models for normalized and non-normalized on training data

In [84]:
# Best not normalized
rf = RandomForestClassifier(labelCol = 'label', featuresCol = 'features', numTrees = 20, maxDepth = 30)
model= rf.fit(normalized_df_train)
predictions = model.transform(normalized_df_test)
df_misclassified_0 = predictions.filter(col('label') == 1).filter(col('prediction') == 0)
df_misclassified_1 = predictions.filter(col('label') == 0).filter(col('prediction') == 1)
df_misclassified = df_misclassified_0.union(df_misclassified_1)
#display(df_misclassified.select('ID','SUMMARY','Pruned Textual Contents','label','prediction'))
df_misclassified.select('ID','SUMMARY','Pruned Textual Contents','label','prediction').show(n=10)

+---------------+--------------------+-----------------------+-----+----------+
|             ID|             SUMMARY|Pruned Textual Contents|label|prediction|
+---------------+--------------------+-----------------------+-----+----------+
|HTTPCLIENT-1051|SSL connections c...|         [connect, use]|  1.0|       0.0|
|HTTPCLIENT-1053|Security issue - ...|            [use, valu]|  1.0|       0.0|
|HTTPCLIENT-1079|Kerberos cross-re...|              [support]|  1.0|       0.0|
|HTTPCLIENT-1116|ResponseCachingPo...|                  [use]|  1.0|       0.0|
| HTTPCLIENT-113|NullPointerExcept...|                     []|  1.0|       0.0|
|HTTPCLIENT-1147|When HttpClient-C...|   [when, cach, cach...|  1.0|       0.0|
| HTTPCLIENT-280|Freezes w MultiTh...|                     []|  1.0|       0.0|
| HTTPCLIENT-289|MultiThreadedHttp...|               [thread]|  1.0|       0.0|
| HTTPCLIENT-292|NTLM Proxy and ba...|                     []|  1.0|       0.0|
| HTTPCLIENT-307|An IOException or...|  

In [85]:
# Best normalized
lsvc = LinearSVC(labelCol = 'label', featuresCol = 'Normalized Features', regParam = 0.001, maxIter = 15)
model= lsvc.fit(normalized_df_train)
predictions = model.transform(normalized_df_test)
df_misclassified_0 = predictions.filter(col('label') == 1).filter(col('prediction') == 0)
df_misclassified_1 = predictions.filter(col('label') == 0).filter(col('prediction') == 1)
df_misclassified = df_misclassified_0.union(df_misclassified_1)
#display(df_misclassified.select('ID','SUMMARY','Pruned Textual Contents','label','prediction'))
df_misclassified.select('ID','SUMMARY','Pruned Textual Contents','label','prediction').show(n=10)

+---------------+--------------------+-----------------------+-----+----------+
|             ID|             SUMMARY|Pruned Textual Contents|label|prediction|
+---------------+--------------------+-----------------------+-----+----------+
|HTTPCLIENT-1051|SSL connections c...|         [connect, use]|  1.0|       0.0|
|HTTPCLIENT-1053|Security issue - ...|            [use, valu]|  1.0|       0.0|
|HTTPCLIENT-1079|Kerberos cross-re...|              [support]|  1.0|       0.0|
|HTTPCLIENT-1116|ResponseCachingPo...|                  [use]|  1.0|       0.0|
| HTTPCLIENT-113|NullPointerExcept...|                     []|  1.0|       0.0|
| HTTPCLIENT-280|Freezes w MultiTh...|                     []|  1.0|       0.0|
| HTTPCLIENT-289|MultiThreadedHttp...|               [thread]|  1.0|       0.0|
| HTTPCLIENT-292|NTLM Proxy and ba...|                     []|  1.0|       0.0|
| HTTPCLIENT-307|An IOException or...|                     []|  1.0|       0.0|
| HTTPCLIENT-350|HttpState matchCr...|  

# 7. Further Analysis

## 7.1 Testing Effect of Using Type Column instead of Classified on Performance

In [70]:
df_combined_processed = preprocess_text(df_all_combined, 2.0, 100)
indexer = StringIndexer(inputCol = 'TYPE', outputCol = 'TypeLabel').setHandleInvalid('skip')
df_result = indexer.fit(df_combined_processed).transform(df_combined_processed)
normalized_df_train, normalized_df_test = df_result.randomSplit([0.8, 0.2], seed = 56)
 
nb = NaiveBayes(labelCol = 'TypeLabel', featuresCol = 'Normalized Features')
paramGrid = ParamGridBuilder().addGrid(nb.smoothing, [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]).build()
 
evaluator = MulticlassClassificationEvaluator(labelCol="TypeLabel", predictionCol="prediction", metricName="f1")
NUM_CV_FOLDS = 2
 
cv = StratifiedCrossValidator(estimator = nb, estimatorParamMaps = paramGrid, evaluator = evaluator, numFolds = NUM_CV_FOLDS)
 
cvModel = cv.fit(normalized_df_train)
 
predictions = cvModel.transform(normalized_df_test)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   72|
|  0.0|       1.0|  461|
|  1.0|       0.0|  361|
|  0.0|       0.0|  286|
+-----+----------+-----+

AUC: 0.400
Accuracy: 0.303
Precision: 0.135
Recall: 0.166
F1 score: 0.149
Weighted F-Measure: 0.314



In [71]:
df_combined_processed = preprocess_text(df_all_combined, 2.0, 100)
indexer = StringIndexer(inputCol = 'TYPE', outputCol = 'TypeLabel').setHandleInvalid('skip')
df_result = indexer.fit(df_combined_processed).transform(df_combined_processed)
normalized_df_train, normalized_df_test = df_result.randomSplit([0.8, 0.2], seed = 56)
 
rf = RandomForestClassifier(labelCol = 'TypeLabel', featuresCol = 'features', numTrees = 40, maxDepth = 30)
model= rf.fit(normalized_df_train)
predictions = model.transform(normalized_df_test)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  156|
|  0.0|       1.0|  601|
|  1.0|       0.0|  277|
|  0.0|       0.0|  146|
+-----+----------+-----+

AUC: 0.203
Accuracy: 0.256
Precision: 0.206
Recall: 0.360
F1 score: 0.262
Weighted F-Measure: 0.254



## 7.2 Results When Running With All Terms

Without Normalization

In [72]:
df_combined_processed = preprocess_text(df_all_combined, 2.0, 10000000000)
normalized_df_train, normalized_df_test = df_result.randomSplit([0.8, 0.2], seed = 56)
 
rf = RandomForestClassifier(labelCol = 'label', featuresCol = 'features', numTrees = 40, maxDepth = 30)
model= rf.fit(normalized_df_train)
predictions = model.transform(normalized_df_test)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  227|
|  0.0|       1.0|   82|
|  1.0|       0.0|  206|
|  0.0|       0.0|  665|
+-----+----------+-----+

AUC: 0.805
Accuracy: 0.756
Precision: 0.735
Recall: 0.524
F1 score: 0.612
Weighted F-Measure: 0.745



In [73]:
lsvc =  LinearSVC(labelCol = 'label', featuresCol = 'features', regParam = 0.010, maxIter = 15)
model= lsvc.fit(normalized_df_train)
predictions = model.transform(normalized_df_test)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  234|
|  0.0|       1.0|   86|
|  1.0|       0.0|  199|
|  0.0|       0.0|  661|
+-----+----------+-----+

AUC: 0.804
Accuracy: 0.758
Precision: 0.731
Recall: 0.540
F1 score: 0.622
Weighted F-Measure: 0.749



In [74]:
nb = NaiveBayes(labelCol = 'label', featuresCol = 'features', smoothing = 0.6)
model= nb.fit(normalized_df_train)
predictions = model.transform(normalized_df_test)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  244|
|  0.0|       1.0|  117|
|  1.0|       0.0|  189|
|  0.0|       0.0|  630|
+-----+----------+-----+

AUC: 0.519
Accuracy: 0.741
Precision: 0.676
Recall: 0.564
F1 score: 0.615
Weighted F-Measure: 0.735



With Normalization

In [75]:
rf = RandomForestClassifier(labelCol = 'label', featuresCol = 'Normalized Features', numTrees = 80, maxDepth = 30)
model= rf.fit(normalized_df_train)
predictions = model.transform(normalized_df_test)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  230|
|  0.0|       1.0|   89|
|  1.0|       0.0|  203|
|  0.0|       0.0|  658|
+-----+----------+-----+

AUC: 0.802
Accuracy: 0.753
Precision: 0.721
Recall: 0.531
F1 score: 0.612
Weighted F-Measure: 0.743



In [76]:
lsvc =  LinearSVC(labelCol = 'label', featuresCol = 'Normalized Features', regParam = 0.001, maxIter = 10)
model= lsvc.fit(normalized_df_train)
predictions = model.transform(normalized_df_test)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  246|
|  0.0|       1.0|  106|
|  1.0|       0.0|  187|
|  0.0|       0.0|  641|
+-----+----------+-----+

AUC: 0.796
Accuracy: 0.752
Precision: 0.699
Recall: 0.568
F1 score: 0.627
Weighted F-Measure: 0.745



In [77]:
nb = NaiveBayes(labelCol = 'label', featuresCol = 'Normalized Features', smoothing = 0.2)
model= nb.fit(normalized_df_train)
predictions = model.transform(normalized_df_test)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  230|
|  0.0|       1.0|  100|
|  1.0|       0.0|  203|
|  0.0|       0.0|  647|
+-----+----------+-----+

AUC: 0.575
Accuracy: 0.743
Precision: 0.697
Recall: 0.531
F1 score: 0.603
Weighted F-Measure: 0.734



## 7.3 Comparing Extraction Techniques

In [86]:
df_combined_processed = preprocess_text(df_all_combined, 'none', 10000000000)
hashingTF = HashingTF(inputCol="Pruned Textual Contents", outputCol="rawFeatures")
featurizedData = hashingTF.transform(df_combined_processed)
idf = IDF(inputCol="rawFeatures", outputCol="TF-IDF")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
rescaledData = rescaledData.withColumnRenamed('features','CountVectorizer').select('SUMMARY','Pruned Textual Contents','label','CountVectorizer','TF-IDF')
word2vec = Word2Vec(vectorSize = 10000, inputCol = 'Pruned Textual Contents', outputCol = 'Word2Vec')
model = word2vec.fit(rescaledData)
result = model.transform(rescaledData)
assembler = VectorAssembler(inputCols = ["CountVectorizer", "TF-IDF","Word2Vec"], outputCol = "CombinedFeatures")
assembled = assembler.transform(result)
#display(assembled)
assembled.show(n=10)

+--------------------+-----------------------+-----+--------------------+--------------------+--------------------+--------------------+
|             SUMMARY|Pruned Textual Contents|label|     CountVectorizer|              TF-IDF|            Word2Vec|    CombinedFeatures|
+--------------------+-----------------------+-----+--------------------+--------------------+--------------------+--------------------+
|HTTPClient 4.1 au...|   [httpclient, auto...|  0.0|(5648,[8,119,398,...|(262144,[34146,44...|[-0.0011841046980...|(277792,[8,119,39...|
|Setting SSLSocket...|   [set, sslsocket, ...|  0.0|(5648,[24,200,473...|(262144,[111743,1...|[-0.0016788046341...|(277792,[24,200,4...|
|Via NTLM proxy to...|   [via, ntlm, proxi...|  0.0|(5648,[53,104,112...|(262144,[11941,33...|[-7.8199575596234...|(277792,[53,104,1...|
|HttpState PREEMPT...|   [httpstate, preem...|  0.0|(5648,[8,23,773,8...|(262144,[85221,12...|[-0.0020131791864...|(277792,[8,23,773...|
|ClientConnectionM...|   [clientconnectio

In [79]:
df_train, df_test = assembled.randomSplit([0.8, 0.2], seed = 1138)
 
# LSVC used for comparison
print('With CountVectorier-------------------------------------')
lsvc =  LinearSVC(labelCol = 'label', featuresCol = 'CountVectorizer', regParam = 0.010, maxIter = 15)
model= lsvc.fit(df_train)
predictions = model.transform(df_test)
print_metrics(predictions)
 
print('With TF-IDF-------------------------------------')
lsvc =  LinearSVC(labelCol = 'label', featuresCol = 'TF-IDF', regParam = 0.010, maxIter = 15)
model= lsvc.fit(df_train)
predictions = model.transform(df_test)
print_metrics(predictions)
 
print('With Word2Vec-------------------------------------')
lsvc =  LinearSVC(labelCol = 'label', featuresCol = 'Word2Vec', regParam = 0.010, maxIter = 15)
model= lsvc.fit(df_train)
predictions = model.transform(df_test)
print_metrics(predictions)
 
print('With All Combined-------------------------------------')
lsvc =  LinearSVC(labelCol = 'label', featuresCol = 'CombinedFeatures', regParam = 0.010, maxIter = 15)
model= lsvc.fit(df_train)
predictions = model.transform(df_test)
print_metrics(predictions)

With CountVectorier-------------------------------------
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  316|
|  0.0|       1.0|  120|
|  1.0|       0.0|  165|
|  0.0|       0.0|  721|
+-----+----------+-----+

AUC: 0.843
Accuracy: 0.784
Precision: 0.725
Recall: 0.657
F1 score: 0.689
Weighted F-Measure: 0.782

With TF-IDF-------------------------------------
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  316|
|  0.0|       1.0|  121|
|  1.0|       0.0|  165|
|  0.0|       0.0|  720|
+-----+----------+-----+

AUC: 0.843
Accuracy: 0.784
Precision: 0.723
Recall: 0.657
F1 score: 0.688
Weighted F-Measure: 0.781

With Word2Vec-------------------------------------
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  346|
|  0.0|       1.0|  263|
|  1.0|       0.0|  135|
|  0.0|       0.0|  578|
+-----+----------+-----+

AUC: 0.758
Accuracy: 0.699
Precision: 

## 7.4 Comparing Against Original Paper Results

In [80]:
df_httpclient = df_issues_original.filter(col("ID").contains("HTTPCLIENT"))
df_lucene = df_issues_original.filter(col("ID").contains("JCR"))
df_jcr = df_issues_original.filter(col("ID").contains("LUCENE"))

In [81]:
df_httpclient_processed = preprocess_text(df_httpclient, 'none', 'all')
df_http_train, df_http_test = df_httpclient_processed.randomSplit([0.7, 0.3], seed = 50)
 
 
nb = NaiveBayes(labelCol = 'label', featuresCol = 'features', smoothing = 0.6)
model = nb.fit(df_http_train)
predictions = model.transform(df_http_test)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|   84|
|  0.0|       1.0|   29|
|  1.0|       0.0|   23|
|  0.0|       0.0|   89|
+-----+----------+-----+

AUC: 0.527
Accuracy: 0.769
Precision: 0.743
Recall: 0.785
F1 score: 0.764
Weighted F-Measure: 0.769



In [82]:
df_lucene_processed = preprocess_text(df_lucene, 'none', 'all')
df_lucene_train, df_lucene_test = df_lucene_processed.randomSplit([0.7, 0.3], seed = 50)
 
 
nb = NaiveBayes(labelCol = 'label', featuresCol = 'features', smoothing = 0.6)
model = nb.fit(df_lucene_train)
predictions = model.transform(df_lucene_test)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  214|
|  0.0|       1.0|   86|
|  1.0|       0.0|   50|
|  0.0|       0.0|  371|
+-----+----------+-----+

AUC: 0.437
Accuracy: 0.811
Precision: 0.713
Recall: 0.811
F1 score: 0.759
Weighted F-Measure: 0.814



In [83]:
df_jcr_processed = preprocess_text(df_jcr, 'none', 'all')
df_jcr_train, df_jcr_test = df_jcr_processed.randomSplit([0.7, 0.3], seed = 50)
 
 
nb = NaiveBayes(labelCol = 'label', featuresCol = 'features', smoothing = 0.6)
model = nb.fit(df_jcr_train)
predictions = model.transform(df_jcr_test)
print_metrics(predictions)

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0|  155|
|  0.0|       1.0|   89|
|  1.0|       0.0|   45|
|  0.0|       0.0|  441|
+-----+----------+-----+

AUC: 0.428
Accuracy: 0.816
Precision: 0.635
Recall: 0.775
F1 score: 0.698
Weighted F-Measure: 0.822

